# Project Introduction:
Primer laboratorio de aprendizaje automático, desarrollado por Sergio Barragán Blanco (100472343) y Eduardo Alarcón Navarro (100472175). 
Grupo 17.


# 1. EDA
Existen 23 carácterísticas que definen cada momento, de las cuales ninguna es categórica, todas son numéricas (con la energía suman 24). No existen valores faltantes, pero si los hubiera, los rellenaríamos con la media del valor superior e inferior antes de randomizar el dataset.

No existen tampoco columnas constantes, que se eliminarían. 

Con todo esto, podemos observar que es un problema de regresión.

La variable que estamos intentando predecir es la "energía" que es el valor de la enrgía generada 24 horas después. 

## Correlaciones entre parámetros:
Al principio, parece que las columnas lai_lv.13 y lai_hv.13 tienen una correlación, pero según avanza el tiempo, desaparece.

## Escala de los datos
Entre las diferentes columnas de datos, tenemos valores y magnitudes muy dispares.

In [24]:
import pandas as pd

# Load the data
data = pd.read_csv('wind_ava.csv.gz', compression='gzip')
# FIlter the data to only include the columns that end in 13
import time
a = time.time()
data = data.filter(regex='13$|energy')
#print(data)
print(time.time()-a)



0.001993894577026367


In [25]:
from sklearn.model_selection import train_test_split

y = data['energy']
x = data.drop(columns=['energy'])


X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=7543)


In [26]:
# Preprocessing

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import Normalizer
from sklearn.compose import ColumnTransformer

scalers = [StandardScaler(), MinMaxScaler(), RobustScaler(), Normalizer()]

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), x.columns)
    ])


In [31]:
# Optimizing hyperparameters

from sklearn.model_selection import GridSearchCV, KFold, RandomizedSearchCV
from sklearn.tree import DecisionTreeRegressor  # Import DecisionTreeRegressor instead of DecisionTreeClassifier
from sklearn.pipeline import Pipeline


param_grid = {
    'criterion': ['friedman_mse', 'absolute_error', 'poisson', 'squared_error'],  # Adjust criterion for regression
    'max_depth': [None, 12, 13, 14, 15, 16, 17],
    'min_samples_split': range(1, 15, 2),
    'min_samples_leaf': range(5, 15, 2),
}

cv = KFold(n_splits=5)
model2 = RandomizedSearchCV(DecisionTreeRegressor(random_state=7543),  # Use DecisionTreeRegressor
                     param_grid, cv=cv, n_jobs=-1, verbose=1, scoring='neg_mean_absolute_percentage_error', n_iter=50)  # Adjust scoring metric

model = GridSearchCV(DecisionTreeRegressor(random_state=7543),  # Use DecisionTreeRegressor
                     param_grid, cv=cv, n_jobs=-1, verbose=1, scoring='neg_mean_absolute_percentage_error')

pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('model', model)
])

pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)

print(f"Best params: {model.best_params_}")
# Best params: {'criterion': 'absolute_error', 'max_depth': 15, 'min_samples_leaf': 11, 'min_samples_split': 3}


Fitting 5 folds for each of 980 candidates, totalling 4900 fits


C:\Users\edual\PycharmProjects\AA-LAB1\venv\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
700 fits failed out of a total of 4900.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
700 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\edual\PycharmProjects\AA-LAB1\venv\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\edual\PycharmProjects\AA-LAB1\venv\Lib\site-packages\sklearn\base.py", line 1467, in wrapper
    estimator._validate_params()
  File "C:\Users\edual\PycharmProjects\AA-LAB1\venv\Lib\site-packages\sklearn\base.py", line 666, in _validate_params
    val

Best params: {'criterion': 'absolute_error', 'max_depth': 15, 'min_samples_leaf': 11, 'min_samples_split': 3}
